# QIIME2 exploratory data analysis workshop

This workshop will guide you through a basic data processing pipeline using QIIME2 on a microbiome dataset. A large proportion will be exploratory data analysis, followed by some more specific statistical analysis and plotting of data.

By-and-large this workshop follows the steps in the [QIIME2 overview tutorial "moving pictures"](https://docs.qiime2.org/2019.7/tutorials/moving-pictures/). There are many more tutorials available on the [QIIME2 tutorial website](https://docs.qiime2.org/2019.7/tutorials/).

## Some basic QIIME2 concepts

For a detailed overview of QIIME2's concepts, [please consult the excellent documentation](https://docs.qiime2.org/2019.7/concepts/).

QIIME2 only handles most "raw" data (such as FASTQ files) on import and bundles data into `.qza` files - QIIME zipped *artifact*. In addition to the actual data, this includes meta-information, such as type of data and provenance.

*Visualisations* on the other hand are wrapped in `.qzv` files - QIIME zipped *visualisation*, with similar meta-information as *artifacts*. *Visualisation* files are usually the terminal output of a process or analysis.

QIIME2's extensive functionality is built with *plugins*. Each *plugin* might wrap specific programmes in other programming languages (such as C or Fortran) and generally fullfills one aspect of data processing or analysis. You can find a [full list of QIIME2 plugins on the website](https://docs.qiime2.org/2019.7/plugins/available/). Throughout this workshop you will also find links to corresponding tutorials.

The structure of QIIME2 commands is always similar, starting with `qiime` followed by the *plugin* name and several flags for options and file input or outputs. You can access the documentation by adding the `--help` flag, e.g.

In [2]:
!qiime tools import --help

Usage: qiime tools import [OPTIONS]

  Import data to create a new QIIME 2 Artifact. See https://docs.qiime2.org/
  for usage examples and details on the file types and associated semantic
  types that can be imported.

Options:
  --type TEXT             The semantic type of the artifact that will be
                          created upon importing. Use --show-importable-types
                          to see what importable semantic types are available
                          in the current deployment.                [required]
  --input-path PATH       Path to file or directory that should be imported.
                                                                    [required]
  --output-path ARTIFACT  Path where output artifact should be written.
                                                                    [required]
  --input-format TEXT     The format of the data to be imported. If not
                          provided, data must be in the format expected by the
     

Note the exclaimation point (`!`) at the beginning of the command. This invokes the command as a `shell` operation rather than the default Python. Alternatively, you can use `%%bash` at the top of the cell to turn the cell into a `shell/bash` script.

You will frequently find `%%time` throughout this workshop, which will give you and idea how long it takes to execute code.

## Viewing *Visualisations* in Jupyter

*Visualisations* can be viewed with a QIIME view plugin, however, as we are working on a supercomputer, a so called 'headless' system without a graphic user interfact. Hence, throughout the analysis, we will do a little work-around using QIIME's `Visualization` Python package (note the American spelling).

We will therefore need to import the package.

In [1]:
from qiime2 import Visualization

*Visualisation* files can then be loaded through a Python command:

```python
Visualization.load('visualisation-file.qzv')
```

This will load the *Visualisation* in the output for the corresponding cell. Each *Visualisation* will also have an "open in new window" link to display the output in a new window/tab.

## The dataset and metadata

You will analyse an oral microbiome dataset. A total of 12 samples were collected by swabing the mouth between 8:00 and 18:00; one before brushing teeth, then hourly. The subject was a 21 year old male with good dental health on a vegan diet.

All metadata for this experiment has been [colated in a spreadsheet](https://docs.google.com/spreadsheets/d/1kdIW4xIxysbf0zQBzO4Y65B9sfaeTZywZ1CrMwWMd9k/edit?usp=sharing). Metadata files follow specific formatting rules (https://docs.qiime2.org/2019.7/tutorials/metadata/) and data included in the files can be used in some of the analyses by QIIME2.

To validate the metadata format, you can either use a QIIME2 plugin, or, more conveniently, use a Google sheets plugin called `Keemei` (Add-ons > Get Add-ons in the menu). Once validated, export the file as a tab-separated value (`.tsv`) file (under File > Download in the menu) and upload it to your working directory (drag and drop onto the files panel of Jupyter).

## I. Importing data 

https://docs.qiime2.org/2019.7/tutorials/importing/

You will work a fraction (~5%) of the total dataset for this particular experiment (about a 1/400th of a full HiSeq run) to reduce computation time.

Let's copy the data to your working directory.

In [ ]:
!cp -rv /home/mbaron/qiime2_workshop/pe-reads/ .

Each of the samples is represented by one forward and one reverse read `.fastq` file - paired end reads. This means the data is already demultiplexed. The `manifest.txt` file in the `pe-reads` folder contains the sample ids and the file paths to each of the files.

To import the data as an *artifact*, `tools import` needs to be informed about the type of data (paired end sequences with quality information), the encoding of the quality scores (can be found in `metadata.yml`) and the filepaths through the manifest.

The import process will take a couple of seconds to a minute and create a `.qza` file.

In [2]:
%%time
!qiime tools import \
    --type 'SampleData[PairedEndSequencesWithQuality]' \
    --input-path manifest3.tsv \
    --output-path demux \
    --input-format PairedEndFastqManifestPhred33V2

Imported manifest3.tsv as PairedEndFastqManifestPhred33V2 to demux
CPU times: user 241 ms, sys: 56.6 ms, total: 298 ms
Wall time: 27.4 s


You can take a brief look at the file type and format using `tools peek`.

In [3]:
%%time
!qiime tools peek demux.qza

UUID:        e63499f0-3a29-44ac-a96b-084d6414316d
Type:        SampleData[PairedEndSequencesWithQuality]
Data format: SingleLanePerSamplePairedEndFastqDirFmt
CPU times: user 9.4 ms, sys: 8.24 ms, total: 17.6 ms
Wall time: 1.1 s


### Visualising demultiplexed data

Once the data is imported, we can visualise the quality score information by creating a `.qzv` file. There are lots of different visualisation methods associated with different `.qza` files (see [available plugins](https://docs.qiime2.org/2019.7/plugins/available/)). `demux summarize` will provide you with an overview of sequences for each sample and an interactive chart displaying quality scores vs sequence length.

Do you see a marked drop in sequence quality?

In [8]:
%%time
!qiime demux summarize \
    --i-data demux.qza \
    --o-visualization demux.qzv

Saved Visualization to: demux.qzv
CPU times: user 266 ms, sys: 53 ms, total: 319 ms
Wall time: 34.1 s


In [2]:
Visualization.load('demux.qzv')

<visualization: Visualization uuid: 917cd5d5-cca3-4339-b7a6-1ac9a9004dd7>

## II. Picking Amplicon Sequence Variants (ASV) with Deblur

The [Deblur](http://msystems.asm.org/content/2/2/e00191-16) algorithm uses error profiles to denoise the sequencing data and find true biological variants. We will quality filter our data by first by joining the paired-end reads. Any remaining low-quality read are removed a Q-score based filtering approach (as described in the lecture).

### Joining paired-end reads

https://docs.qiime2.org/2019.7/plugins/available/vsearch/join-pairs/

The expected size of the region amplified by the 515f and 806r primers is close to 250bp. As the data was generated by a HiSeq Rapid-Run with 250bp reads, the overlap between forward and reverse sequences is likely substaintial. `-p-minovlen` describes the minimum overlap between the reads, while `-p-maxdiffs` sets how many differences are tolerated. The default settings of the `vsearch` algorithm are rather lax, so the overlap was set to a minimum of 50bp (which would allow a total length of 450bp); no differences were tolerated.

In [7]:
%%time
!qiime vsearch join-pairs \
    --i-demultiplexed-seqs demux.qza \
    --output-dir joined2 \
    --p-minovlen 50 \
    --p-maxdiffs 0 \
    --p-allowmergestagger

Saved SampleData[JoinedSequencesWithQuality] to: joined2/joined_sequences.qza
CPU times: user 275 ms, sys: 63.7 ms, total: 339 ms
Wall time: 36.6 s


## /joined to /joinedold, new /joined2 to /joined. Applied to all old dir

In [9]:
%%time
!qiime demux summarize \
    --i-data joined/joined_sequences.qza \
    --o-visualization joined/joined_sequences.qzv

Saved Visualization to: joined/joined_sequences.qzv
CPU times: user 105 ms, sys: 39.5 ms, total: 145 ms
Wall time: 13.7 s


In [3]:
Visualization.load('joined/joined_sequences.qzv')

<visualization: Visualization uuid: 3e050b1c-7a70-4f18-ada2-b884260702bb>

### Quality filtering by Q-score

https://docs.qiime2.org/2019.7/plugins/available/quality-filter/q-score-joined/

In [11]:
%%time
!qiime quality-filter q-score-joined \
    --i-demux joined/joined_sequences.qza \
    --output-dir filtered

Saved SampleData[JoinedSequencesWithQuality] to: filtered/filtered_sequences.qza
Saved QualityFilterStats to: filtered/filter_stats.qza
CPU times: user 667 ms, sys: 164 ms, total: 831 ms
Wall time: 1min 16s


In [12]:
%%time
!qiime demux summarize \
    --i-data filtered/filtered_sequences.qza \
    --o-visualization filtered/filtered_sequences.qzv

Saved Visualization to: filtered/filtered_sequences.qzv
CPU times: user 88 ms, sys: 26.6 ms, total: 115 ms
Wall time: 12.9 s


In [4]:
Visualization.load('filtered/filtered_sequences.qzv')

<visualization: Visualization uuid: 9c4fb04d-d624-4caa-ad6a-3f34e9be8e5a>

In [14]:
%%time
!qiime metadata tabulate \
  --m-input-file filtered/filter_stats.qza \
  --o-visualization filtered/filter_stats.qzv

Saved Visualization to: filtered/filter_stats.qzv
CPU times: user 51.2 ms, sys: 14.9 ms, total: 66.2 ms
Wall time: 5.61 s


In [5]:
Visualization.load('filtered/filter_stats.qzv')

<visualization: Visualization uuid: db65356c-0ba0-4a87-b665-cadf0eaf04f5>

### Deblur

https://docs.qiime2.org/2019.7/plugins/available/deblur/denoise-16S/

With larger datasets, the actual Deblur step can be very time-consuming. With the current relatively small dataset, it should complete with a few minutes on the supercomputer.

This script can run in parallel on several CPU cores by using the `--p-jobs-to-start` flag. You will come across other plugins, which can also be parallelised, though the flags might be called differently, containg works like `threads` or `cores`. The majority of Cartesius compute nodes have 24 cores available.

The output is a feature table, statistics on the denoising and ASV-picking process as well as a file only containing key representative sequences (ASVs). The representative sequences, as a smaller file, allow faster construction of a phylogenetic tree or assignment of taxonomy.

In [17]:
%%time
!qiime deblur denoise-16S \
    --i-demultiplexed-seqs filtered/filtered_sequences.qza \
    --p-trim-length 251 \
    --output-dir deblur \
    --p-sample-stats \
    --p-jobs-to-start 24

Saved FeatureTable[Frequency] to: deblur/table.qza
Saved FeatureData[Sequence] to: deblur/representative_sequences.qza
Saved DeblurStats to: deblur/stats.qza
CPU times: user 895 ms, sys: 176 ms, total: 1.07 s
Wall time: 1min 44s


In [18]:
%%time
!qiime deblur visualize-stats \
    --i-deblur-stats deblur/stats.qza \
    --o-visualization deblur/stats.qzv

Saved Visualization to: deblur/stats.qzv
CPU times: user 45.7 ms, sys: 18.4 ms, total: 64.1 ms
Wall time: 5.61 s


In [2]:
Visualization.load('deblur/stats.qzv')

<visualization: Visualization uuid: 48b69b17-5239-40b2-b58b-d1f25a7ebcb6>

### deblur table

In [8]:
%%time
!qiime feature-table summarize \
  --i-table deblur/table.qza \
  --o-visualization deblur/table.qzv \
  --m-sample-metadata-file phones_meta3.tsv

Saved Visualization to: deblur/table.qzv
CPU times: user 301 ms, sys: 79 ms, total: 380 ms
Wall time: 24.9 s


In [20]:
Visualization.load('deblur/table.qzv')

<visualization: Visualization uuid: 224e71fe-c4d3-4ab8-afc5-1a9d667d6db5>

## III. Assigning taxonomy using the GG (http://www.homd.org/index.php)

https://docs.qiime2.org/2019.7/tutorials/feature-classifier/

There are many references databases, some of the more popular general references, such as GreenGenes and Silva, can be found through the [QIIME data resources page](https://docs.qiime2.org/2019.7/data-resources/).

Using a more specialised a database has the advantage of more reliable and accurate taxonomy assignment. More specialised databases are usually smaller and allowing faster processing.

### Importing GG data

Two files are required to build a taxonomy classifier, the references sequences in `.fasta` format and text-file file liking the reference IDs with taxa. Both these files can be found on [HOMD ftp download pages for their 16S rRNA references](http://www.homd.org/ftp/16S_rRNA_refseq/HOMD_16S_rRNA_RefSeq/V15.2/). Let's make a new directory and download them directly using `wget`.

In [9]:
!mkdir gg
!wget -P gg "https://gg-sg-web.s3-us-west-2.amazonaws.com/downloads/greengenes_database/gg_13_5/gg_13_5.fasta.gz"
!wget -P gg "https://gg-sg-web.s3-us-west-2.amazonaws.com/downloads/greengenes_database/gg_13_5/gg_13_5_taxonomy.txt.gz"

--2020-02-25 23:51:19--  https://gg-sg-web.s3-us-west-2.amazonaws.com/downloads/greengenes_database/gg_13_5/gg_13_5.fasta.gz
Resolving gg-sg-web.s3-us-west-2.amazonaws.com (gg-sg-web.s3-us-west-2.amazonaws.com)... 52.218.232.185
Connecting to gg-sg-web.s3-us-west-2.amazonaws.com (gg-sg-web.s3-us-west-2.amazonaws.com)|52.218.232.185|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 272134013 (260M) [binary/octet-stream]
Saving to: ‘gg/gg_13_5.fasta.gz’

100%[======================================>] 272,134,013 9.62MB/s   in 30s    

2020-02-25 23:51:52 (8.64 MB/s) - ‘gg/gg_13_5.fasta.gz’ saved [272134013/272134013]

--2020-02-25 23:51:52--  https://gg-sg-web.s3-us-west-2.amazonaws.com/downloads/greengenes_database/gg_13_5/gg_13_5_taxonomy.txt.gz
Resolving gg-sg-web.s3-us-west-2.amazonaws.com (gg-sg-web.s3-us-west-2.amazonaws.com)... 52.218.204.225
Connecting to gg-sg-web.s3-us-west-2.amazonaws.com (gg-sg-web.s3-us-west-2.amazonaws.com)|52.218.204.225|:443... conn

For importing, `tools import` does the job. Note that we specifying different `--type` for each of the imports.

In [15]:
!wget -P gg "https://gg-sg-web.s3-us-west-2.amazonaws.com/downloads/greengenes_database/gg_13_5/gg_13_5_otus.tar.gz"

--2020-02-25 23:59:13--  https://gg-sg-web.s3-us-west-2.amazonaws.com/downloads/greengenes_database/gg_13_5/gg_13_5_otus.tar.gz
Resolving gg-sg-web.s3-us-west-2.amazonaws.com (gg-sg-web.s3-us-west-2.amazonaws.com)... 52.218.235.57
Connecting to gg-sg-web.s3-us-west-2.amazonaws.com (gg-sg-web.s3-us-west-2.amazonaws.com)|52.218.235.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 318327264 (304M) [application/x-tar]
Saving to: ‘gg/gg_13_5_otus.tar.gz’

100%[======================================>] 318,327,264 7.07MB/s   in 40s    

2020-02-25 23:59:54 (7.58 MB/s) - ‘gg/gg_13_5_otus.tar.gz’ saved [318327264/318327264]



In [16]:
from shutil import unpack_archive 
unpack_archive('gg/gg_13_5_otus.tar.gz', 'gg')

In [ ]:
#unzip and upload .fasta from user
scp ROUTE cartesius:~

In [22]:
from zipfile import ZipFile

In [24]:
%%time
# importing gg data
!qiime tools import \
    --type 'FeatureData[Sequence]' \
    --input-path gg/rep_set/99_otus.fasta \
    --output-path gg/gg_1

Imported gg//rep_set/99_otus.fasta as DNASequencesDirectoryFormat to gg/gg_1
CPU times: user 256 ms, sys: 70.5 ms, total: 326 ms
Wall time: 44.4 s


In [25]:
%%time
!qiime tools import \
    --type 'FeatureData[Taxonomy]' \
    --input-format HeaderlessTSVTaxonomyFormat \
    --input-path gg/taxonomy/99_otu_taxonomy.txt \
    --output-path gg/gg.qiime.taxonomy

Imported gg/taxonomy/99_otu_taxonomy.txt as HeaderlessTSVTaxonomyFormat to gg/gg.qiime.taxonomy
CPU times: user 41.9 ms, sys: 14.2 ms, total: 56.1 ms
Wall time: 5.15 s


### Trimming references with primer sequences

https://docs.qiime2.org/2019.7/plugins/available/feature-classifier/extract-reads/

Trimming the reference database to the region of interest (in our case between 515f and 806r), also ensure more reliable assignment of taxonomy. To do so, the plugin requires the annealing regions of both primer sequences (note the degenerate bases e.g `M`) and minimum and maximum expected sequence lengths. As the region should be around 250 bp, a minimum length of 100 bp and a maximum length of 400 bp should reduce spurious sequences.

In [26]:
%%time
## extracting reads according to primers
!qiime feature-classifier extract-reads \
    --i-sequences gg/gg_1.qza \
    --p-f-primer GTGCCAGCMGCCGCGGTAA \
    --p-r-primer GGACTACHVGGGTWTCTAAT \
    --p-min-length 100 \
    --p-max-length 400 \
    --o-reads gg/gg_1_trimmed.qza

Saved FeatureData[Sequence] to: gg/gg_1_trimmed.qza
CPU times: user 6.42 s, sys: 1.18 s, total: 7.59 s
Wall time: 13min 9s


### Assigning taxonomy

https://docs.qiime2.org/2019.7/plugins/available/feature-classifier/classify-consensus-vsearch/

VSEARCH is used to align our query sequences (remember the representative sequences generated by Deblur?) with the reference and assign taxonomy.

In [5]:
%%time
!qiime feature-classifier classify-consensus-vsearch \
    --i-query deblur/representative_sequences.qza \
    --i-reference-reads gg/gg_1_trimmed.qza \
    --i-reference-taxonomy gg/gg.qiime.taxonomy.qza \
    --output-dir taxonomy \
    --p-threads 24

Saved FeatureData[Taxonomy] to: taxonomy/classification.qza
CPU times: user 137 ms, sys: 43.2 ms, total: 180 ms
Wall time: 16.5 s


### Taxa bar-plot

https://docs.qiime2.org/2019.7/plugins/available/taxa/barplot/

A convenient way to visualise taxonomy is through stacked bar-charts. Do you see any trends in any taxa throughout the day? We'll get back to the taxonomy data later.

In [11]:
%%time
!qiime taxa barplot \
    --i-table deblur/table.qza \
    --i-taxonomy taxonomy/classification.qza \
    --m-metadata-file phones_meta2.tsv \
    --o-visualization taxonomy/bar-plot2.qzv

Saved Visualization to: taxonomy/bar-plot2.qzv
CPU times: user 235 ms, sys: 64 ms, total: 299 ms
Wall time: 21.4 s


In [2]:
Visualization.load('taxonomy/bar-plot2.qzv')

<visualization: Visualization uuid: c3772f88-d205-4670-90f9-75041de4658b>

## Generating a phylogenetic tree

https://docs.qiime2.org/2019.7/plugins/available/phylogeny/align-to-tree-mafft-fasttree/

Several phylogeny-based diversity metrics require a phylogenetic tree. This pipeline method will align all the sequences, denoise the data and construct both a rooted and unrooted phylogenetic tree. Again, using only the representative sequences speeds up this process.

In [30]:
%%time
!qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences deblur/representative_sequences.qza \
    --p-n-threads 24 \
    --output-dir phylogeny

Saved FeatureData[AlignedSequence] to: phylogeny/alignment.qza
Saved FeatureData[AlignedSequence] to: phylogeny/masked_alignment.qza
Saved Phylogeny[Unrooted] to: phylogeny/tree.qza
Saved Phylogeny[Rooted] to: phylogeny/rooted_tree.qza
CPU times: user 143 ms, sys: 43.8 ms, total: 186 ms
Wall time: 19 s


### Visualise phylogentic tree

QIIME2 doesn't yet have plugins to visualise phylogenetic trees. If you like to see the tree, download your `phylogeny/tree.qza` file (right-click on the file in the file panel and download) and upload it to [Interactive Tree Of Life](https://itol.embl.de/upload.cgi), a tool by maintained by the EMBL.

For full annotation of the leafs, also download `taxonomy/classification.qza` and drag and drop the file onto the tree. The [iTOL help pages list several other QIIME2 artifacts which can be applied to the tree](https://itol.embl.de/help.cgi#qiime).

## IV. Alpha and beta diversity at depth 412 (4.77% features ans 26/30 individuals)
# Abandoned, see V first

https://docs.qiime2.org/2019.7/plugins/available/diversity/core-metrics-phylogenetic/

The core metrics pipeline is a handy tool to generate a whole array of alpha and beta diversity metrics. [This excellent post on the QIIME2 forum](https://forum.qiime2.org/t/alpha-and-beta-diversity-explanations-and-commands/2282) gives a brief overview with papers for each available metric. As mentioned in the lecture, there are many metrics, many of which originating form the much older field of echology. Given that we work with sequencing data you might want to focus at first on phylogenetic metrics, such as [Faith's Phylogenetic Distance](https://en.wikipedia.org/wiki/Phylogenetic_diversity) for alpha diversity and UniFrac for beta diversity.

The pipeline required a rooted phylogenetic tree, the feature table, the meta-data file, as well as a sampling-depth parameter. Each sample will be randomly subsampled (rarefied) to the depth of `--p-sampling-depth` to ensure even representation. Any sample smaller than the sampling depth will be excluded. Consequently, this is a tradeoff between depth of the analysis and number of samples retained.

You need to check your visualisation of the Deblur-statistics for sampling depth. Sort the data by the `read-hit-reference`. Is there a large difference between the most and least numerous sample?

In [16]:
%%time
!qiime diversity core-metrics-phylogenetic \
    --i-phylogeny phylogeny/rooted_tree.qza \
    --i-table deblur/table.qza \
    --p-sampling-depth 412 \
    --m-metadata-file phones_meta3.tsv \
    --output-dir alpha_beta \
    --p-n-jobs 24

Saved FeatureTable[Frequency] to: alpha_beta/rarefied_table.qza
Saved SampleData[AlphaDiversity] % Properties('phylogenetic') to: alpha_beta/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: alpha_beta/observed_otus_vector.qza
Saved SampleData[AlphaDiversity] to: alpha_beta/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: alpha_beta/evenness_vector.qza
Saved DistanceMatrix % Properties('phylogenetic') to: alpha_beta/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix % Properties('phylogenetic') to: alpha_beta/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: alpha_beta/jaccard_distance_matrix.qza
Saved DistanceMatrix to: alpha_beta/bray_curtis_distance_matrix.qza
Saved PCoAResults to: alpha_beta/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: alpha_beta/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: alpha_beta/jaccard_pcoa_results.qza
Saved PCoAResults to: alpha_beta/bray_curtis_pcoa_results.qza
Saved Visualization to: alpha_beta/un

[Ordination](https://en.wikipedia.org/wiki/Ordination_(statistics)) is a popular tool to explore microbial communities. We can use the [Emperor tool](https://biocore.github.io/emperor/description_index.html) to inspect [principal coordinate analysis (PCoA)](https://moodle.ucl.ac.uk/mod/page/view.php?id=988175) plots. Due to a bug in the QIIME2 implementations data labels can not be set, however when you click on a sample point, its label is revealed in the bottom left corner of the plot.

#### Jaccard emperor plot

In [2]:
Visualization.load('alpha_beta/jaccard_emperor.qzv')

<visualization: Visualization uuid: b07b3fed-def5-434d-bf54-ac756d051881>

### Alpha diversity correlation

https://docs.qiime2.org/2019.7/plugins/available/diversity/alpha-correlation/

Given that this dataset is a time-series, it makes sense to look for correlation over time. The alpha-correlation method will determine whether any numeric data provided in the metadata file are correlated with alpha diversity. By default it will apply a [Spearman's rank correlation test](https://statistics.laerd.com/spss-tutorials/spearmans-rank-order-correlation-using-spss-statistics.php), generate a correlation coefficient (-1 to 1) and a p-value. Is the correlation significant?

In [17]:
%%time
!qiime diversity alpha-correlation \
  --i-alpha-diversity alpha_beta/faith_pd_vector.qza \
  --m-metadata-file phones_meta3.tsv \
  --o-visualization alpha_beta/faith_pd_correlation

Saved Visualization to: alpha_beta/faith_pd_correlation.qzv
CPU times: user 101 ms, sys: 26.7 ms, total: 128 ms
Wall time: 9.18 s


In [4]:
Visualization.load('alpha_beta/faith_pd_correlation.qzv')

<visualization: Visualization uuid: c86f2069-e8fd-438a-812b-dad9b2ba4be9>

### Alpha rarefaction curves

https://docs.qiime2.org/2019.7/plugins/available/diversity/alpha-rarefaction/

Rarefaction curves are generated by randomly sub-sampling data from each sample at increasing depth (up to `-p-max-depth`). Each sub-sampling is repeated several times (default: 10) and alpha diversity metric are calculated. As the sampling depth increases, so should the diversity indices. If the curves plateau or level out, increasing sequencing depth would unlikely increase the number of features deteced.

Hence, alpha rarefaction curves are a tool to determine whether most of the diversity had been captured through sequencing.

In [20]:
%%time
!qiime diversity alpha-rarefaction \
    --i-table deblur/table.qza \
    --i-phylogeny phylogeny/rooted_tree.qza \
    --p-max-depth 420 \
    --p-metrics chao1 \
    --p-metrics faith_pd \
    --p-metrics shannon \
    --p-metrics observed_otus \
    --m-metadata-file phones_meta3.tsv \
    --p-steps 20 \
    --o-visualization alpha_beta/rarefaction420

Saved Visualization to: alpha_beta/rarefaction420.qzv
CPU times: user 554 ms, sys: 141 ms, total: 695 ms
Wall time: 1min


In [2]:
Visualization.load('alpha_beta/rarefaction420.qzv')

<visualization: Visualization uuid: 14dbb2cc-86c9-45b9-9700-1ba4e409fc35>

### Visualisations

#### oberserved otus 

In [23]:
%%time
!qiime diversity alpha-group-significance \
    --i-alpha-diversity alpha_beta/observed_otus_vector.qza \
    --m-metadata-file phones_meta3.tsv \
    --o-visualization AGS/observed_otus_vector.qzv

Saved Visualization to: AGS/observed_otus_vector.qzv
CPU times: user 74.6 ms, sys: 23.8 ms, total: 98.4 ms
Wall time: 8.07 s


In [24]:
 Visualization.load('AGS/observed_otus_vector.qzv')

<visualization: Visualization uuid: a6613195-2ea9-43f6-a992-37129bd857ec>

#### Shannon index for abundance and evenness

In [25]:
%%time
!qiime diversity alpha-group-significance \
    --i-alpha-diversity alpha_beta/shannon_vector.qza \
    --m-metadata-file phones_meta3.tsv \
    --o-visualization AGS/shannon_vector.qzv

Saved Visualization to: AGS/shannon_vector.qzv
CPU times: user 88.4 ms, sys: 17.9 ms, total: 106 ms
Wall time: 8.6 s


In [26]:
Visualization.load('AGS/shannon_vector.qzv')

<visualization: Visualization uuid: 40a2be7f-35db-4a6c-b67d-aa2af9ae1a91>

#### Faith PD ( faith’s phylogenetic diversity—measures of biodiversity that incorporates phylogenetic difference between species)

In [29]:
%%time
!qiime diversity alpha-group-significance \
    --i-alpha-diversity alpha_beta/faith_pd_vector.qza \
    --m-metadata-file phones_meta3.tsv \
    --o-visualization AGS/faith_pd_vector.qzv

Saved Visualization to: AGS/faith_pd_vector.qzv
CPU times: user 67.6 ms, sys: 23.2 ms, total: 90.8 ms
Wall time: 8.02 s


In [14]:
Visualization.load('AGS/faith_pd_vector.qzv')

<visualization: Visualization uuid: eb92ba2f-53ce-4750-bbf0-4c6d87232ed1>

#### Chao1 index (Estimates diversity from abundant data)
##### N.B. This metric is independent of sampling depth

In [27]:
!qiime diversity alpha \
  --i-table deblur/table.qza \
  --p-metric chao1 \
  --o-alpha-diversity alpha_beta/chao1_index.qza

Saved SampleData[AlphaDiversity] to: alpha_beta/chao1_index.qza


In [27]:
%%time
!qiime diversity alpha-group-significance \
    --i-alpha-diversity alpha_beta/chao1_index.qza \
    --m-metadata-file phones_meta3.tsv \
    --o-visualization AGS/chao1_index.qzv

Saved Visualization to: AGS/chao1_index.qzv
CPU times: user 59.1 ms, sys: 22.3 ms, total: 81.3 ms
Wall time: 6.96 s


In [15]:
Visualization.load('AGS/chao1_index.qzv')

<visualization: Visualization uuid: 87fababe-0c5b-4064-82b1-b4b5b97dda2e>

## Beta diversity
### UniFrac
#### Hand or phone

In [30]:
%%time
!qiime diversity beta-group-significance \
  --i-distance-matrix alpha_beta/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file phones_meta3.tsv \
  --m-metadata-column 'hand or phone' \
  --o-visualization BGS/unweighted_unifrac_subject_group_significance_hop.qzv \
  --p-pairwise


Saved Visualization to: BGS/unweighted_unifrac_subject_group_significance_hop.qzv
CPU times: user 77 ms, sys: 27.4 ms, total: 104 ms
Wall time: 7.95 s


In [8]:
Visualization.load('BGS/unweighted_unifrac_subject_group_significance_hop.qzv')

<visualization: Visualization uuid: 20492c97-be21-4e04-8ea9-af9573e638f9>

#### frequency of phone cleanage 

In [34]:
%%time
!qiime diversity beta-group-significance \
  --i-distance-matrix alpha_beta/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file phones_meta3.tsv \
  --m-metadata-column 'frequency of phone cleanage' \
  --o-visualization BGS/unweighted_unifrac_subject_group_significance_freq.qzv \
  --p-pairwise


Saved Visualization to: BGS/unweighted_unifrac_subject_group_significance_freq.qzv
CPU times: user 83.9 ms, sys: 30.8 ms, total: 115 ms
Wall time: 9.02 s


In [32]:
Visualization.load('BGS/unweighted_unifrac_subject_group_significance_freq.qzv')

<visualization: Visualization uuid: 38543883-758f-49d1-b80f-02741ac86815>

#### gender

In [37]:
%%time
!qiime diversity beta-group-significance \
  --i-distance-matrix alpha_beta/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file phones_meta3.tsv \
  --m-metadata-column gender \
  --o-visualization BGS/unweighted_unifrac_subject_group_significance_g.qzv \
  --p-pairwise


Saved Visualization to: BGS/unweighted_unifrac_subject_group_significance_g.qzv
CPU times: user 53 ms, sys: 17.9 ms, total: 70.9 ms
Wall time: 6.92 s


In [16]:
Visualization.load('BGS/unweighted_unifrac_subject_group_significance_g.qzv')

<visualization: Visualization uuid: 5a2de655-d219-4ec3-a2cc-59bf10501123>

#### Bray curtis

In [17]:
%%time
!qiime diversity beta-group-significance \
 --i-distance-matrix alpha_beta/bray_curtis_distance_matrix.qza \
 --m-metadata-file "phones_meta3.tsv" \
 --m-metadata-column 'individual' \
 --o-visualization BGS/bray_curtis_distance_matrixind.qzv \

Saved Visualization to: BGS/bray_curtis_distance_matrixind.qzv
CPU times: user 274 ms, sys: 72.8 ms, total: 347 ms
Wall time: 31.3 s


In [2]:
Visualization.load('BGS/bray_curtis_distance_matrixind.qzv')

<visualization: Visualization uuid: 71c4e916-33a0-4221-9792-8d50f9755604>

#### Jaccard similarity index

In [3]:
%%time
!qiime diversity beta-rarefaction \
--i-table deblur/table.qza \
--p-metric jaccard \
--p-clustering-method nj \
--m-metadata-file phones_meta3.tsv \
--p-sampling-depth 412 \
--i-phylogeny phylogeny/rooted_tree.qza \
--o-visualization beta_rarefaction/jaccard.qzv

Saved Visualization to: beta_rarefaction/jaccard.qzv
CPU times: user 125 ms, sys: 29.7 ms, total: 155 ms
Wall time: 14 s


In [4]:
Visualization.load('beta_rarefaction/jaccard.qzv')

<visualization: Visualization uuid: adfb6502-6e6c-4e13-8c21-dede4a2bedd0>

In [25]:
%%time
!qiime diversity filter-distance-matrix \
 --i-distance-matrix alpha_beta/jaccard_distance_matrix.qza \
 --m-metadata-file phones_meta3.tsv \
 --p-where [individual]='2' \
 --o-filtered-distance-matrix BGS/Jaccard/j2.qza

Saved DistanceMatrix to: BGS/Jaccard/j2.qza
CPU times: user 43.2 ms, sys: 20.6 ms, total: 63.8 ms
Wall time: 5.84 s


In [5]:
Visualization.load('BGS/jaccard-significance2.qzv')

<visualization: Visualization uuid: 5b2d8de9-243f-4e53-97eb-5f06b9c99876>

# V. Rerun alpha and beta diversity at sample depth 1014 (10.05% features and 18/30 individuals)

### Filtering

In [22]:
%%time
!qiime feature-table filter-samples \
  --i-table deblur/table.qza \
  --m-metadata-file phones_meta3.tsv \
  --p-where "[individual]= 2 OR [individual]= 11 OR [individual]= 13 OR [individual]= 15 OR [individual]= 16 OR [individual]= 17 OR [individual]= 18 OR [individual]= 19 OR [individual]= 20 OR [individual]= 21 OR [individual]= 22 OR [individual]= 23 OR [individual]= 24 OR [individual]= 26 OR [individual]= 27 OR [individual]= 28 OR [individual]= 29 OR [individual]= 30" \
  --o-filtered-table filtering_sample/table1044.qza

Saved FeatureTable[Frequency] to: filtering_sample/table1044.qza
CPU times: user 50.7 ms, sys: 14.1 ms, total: 64.8 ms
Wall time: 5.38 s


In [24]:
%%time
!qiime feature-table summarize \
  --i-table filtering_sample/table1044.qza \
  --o-visualization filtering_sample/table1044.qzv \
  --m-sample-metadata-file phones_meta3.tsv

Saved Visualization to: filtering_sample/table1044.qzv
CPU times: user 47.4 ms, sys: 14.4 ms, total: 61.8 ms
Wall time: 6.16 s


In [25]:
Visualization.load('filtering_sample/table1044.qzv')

<visualization: Visualization uuid: 44e44634-07b8-4f5c-aea3-0cad98450243>

### rarefaction

In [26]:
%%time
!qiime diversity alpha-rarefaction \
    --i-table filtering_sample/table1044.qza \
    --i-phylogeny phylogeny/rooted_tree.qza \
    --p-max-depth 1100 \
    --p-metrics chao1 \
    --p-metrics faith_pd \
    --p-metrics shannon \
    --p-metrics observed_otus \
    --m-metadata-file phones_meta3.tsv \
    --p-steps 20 \
    --o-visualization filtering_sample/rarefaction1k

Saved Visualization to: filtering_sample/rarefaction1k.qzv
CPU times: user 250 ms, sys: 58 ms, total: 308 ms
Wall time: 41.5 s


In [3]:
Visualization.load('filtering_sample/rarefaction1k.qzv')

<visualization: Visualization uuid: 41d5b1d9-5c46-4a74-8d85-0f08809aac47>

In [28]:
%%time
!qiime diversity core-metrics-phylogenetic \
    --i-phylogeny phylogeny/rooted_tree.qza \
    --i-table filtering_sample/table1044.qza \
    --p-sampling-depth 1014 \
    --m-metadata-file phones_meta3.tsv \
    --output-dir alpha_beta1014 \
    --p-n-jobs 24

Saved FeatureTable[Frequency] to: alpha_beta1014/rarefied_table.qza
Saved SampleData[AlphaDiversity] % Properties('phylogenetic') to: alpha_beta1014/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: alpha_beta1014/observed_otus_vector.qza
Saved SampleData[AlphaDiversity] to: alpha_beta1014/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: alpha_beta1014/evenness_vector.qza
Saved DistanceMatrix % Properties('phylogenetic') to: alpha_beta1014/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix % Properties('phylogenetic') to: alpha_beta1014/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: alpha_beta1014/jaccard_distance_matrix.qza
Saved DistanceMatrix to: alpha_beta1014/bray_curtis_distance_matrix.qza
Saved PCoAResults to: alpha_beta1014/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: alpha_beta1014/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: alpha_beta1014/jaccard_pcoa_results.qza
Saved PCoAResults to: alpha_beta1014/bray_curtis_pc

In [29]:
Visualization.load('alpha_beta1014/jaccard_emperor.qzv')

<visualization: Visualization uuid: 0c8df981-c56f-4d0c-a090-8f0a58ea9abe>

In [30]:
%%time
!qiime diversity alpha-correlation \
  --i-alpha-diversity alpha_beta1014/faith_pd_vector.qza \
  --m-metadata-file phones_meta3.tsv \
  --o-visualization alpha_beta1014/faith_pd_correlation

Saved Visualization to: alpha_beta1014/faith_pd_correlation.qzv
CPU times: user 69 ms, sys: 20.6 ms, total: 89.5 ms
Wall time: 7.39 s


In [31]:
Visualization.load('alpha_beta1014/faith_pd_correlation.qzv')

<visualization: Visualization uuid: 706d4588-9a96-41ec-9f6e-31fe1eff3018>

### Alpha group significance

#### Shannon

In [38]:
%%time
!qiime diversity alpha-group-significance \
    --i-alpha-diversity alpha_beta1014/shannon_vector.qza \
    --m-metadata-file phones_meta3.tsv \
    --o-visualization AGS1014/shannon_vector.qzv

Saved Visualization to: AGS1014/shannon_vector.qzv
CPU times: user 54 ms, sys: 22 ms, total: 76 ms
Wall time: 6.47 s


In [34]:
Visualization.load('AGS1014/shannon_vector.qzv')

<visualization: Visualization uuid: f8423ebf-308c-4363-87b5-2cf4ab037928>

#### Observed OTUs

In [35]:
%%time
!qiime diversity alpha-group-significance \
    --i-alpha-diversity alpha_beta1014/observed_otus_vector.qza \
    --m-metadata-file phones_meta3.tsv \
    --o-visualization AGS1014/observed_otus_vector.qzv

Saved Visualization to: AGS1014/observed_otus_vector.qzv
CPU times: user 49.7 ms, sys: 15.4 ms, total: 65.2 ms
Wall time: 6.4 s


In [ ]:
Visualization.load('AGS1014/observed_otus_vector.qzv')

#### Chao1

In [39]:
%%time
!qiime diversity alpha \
  --i-table filtering_sample/table1044.qza \
  --p-metric chao1 \
  --o-alpha-diversity alpha_beta1014/chao1_index.qza

Saved SampleData[AlphaDiversity] to: alpha_beta1014/chao1_index.qza
CPU times: user 28.7 ms, sys: 15.2 ms, total: 43.8 ms
Wall time: 4.41 s


In [40]:
%%time
!qiime diversity alpha-group-significance \
    --i-alpha-diversity alpha_beta1014/chao1_index.qza \
    --m-metadata-file phones_meta3.tsv \
    --o-visualization AGS1014/chao1_index.qzv

Saved Visualization to: AGS1014/chao1_index.qzv
CPU times: user 42.7 ms, sys: 12.2 ms, total: 55 ms
Wall time: 5.43 s


In [41]:
Visualization.load('AGS1014/chao1_index.qzv')

<visualization: Visualization uuid: 95efa6a8-5fad-4a2d-ab1e-497073b57d20>

#### Faith PD

In [42]:
%%time
!qiime diversity alpha-group-significance \
    --i-alpha-diversity alpha_beta1014/faith_pd_vector.qza \
    --m-metadata-file phones_meta3.tsv \
    --o-visualization AGS1014/faith_pd_vector.qzv

Saved Visualization to: AGS1014/faith_pd_vector.qzv
CPU times: user 51.2 ms, sys: 17.9 ms, total: 69.1 ms
Wall time: 6.58 s


In [43]:
Visualization.load('AGS1014/faith_pd_vector.qzv')

<visualization: Visualization uuid: 8a2d210a-fc94-4ed3-9544-65268bea326a>

## Beta group significance

### Jaccard similarity index  

#### PCoA plot

In [ ]:
Visualization.load('alpha_beta1014/jaccard_emperor.qzv')

#### Gender

In [7]:
%%time
!qiime diversity beta-group-significance \
  --i-distance-matrix alpha_beta1014/jaccard_distance_matrix.qza \
  --m-metadata-file phones_meta3.tsv \
  --m-metadata-column 'gender' \
  --o-visualization BGS1014/jaccard_gender.qzv \
  --p-pairwise

Saved Visualization to: BGS1014/jaccard_gender.qzv
CPU times: user 88.6 ms, sys: 33.3 ms, total: 122 ms
Wall time: 9.99 s


In [8]:
Visualization.load('BGS1014/jaccard_gender.qzv')

<visualization: Visualization uuid: c2d9c8c7-b037-4a0d-b6f6-b5f0bd49fcf2>

#### Frequency of phone-cleaning

In [9]:
%%time
!qiime diversity beta-group-significance \
  --i-distance-matrix alpha_beta1014/jaccard_distance_matrix.qza \
  --m-metadata-file phones_meta3.tsv \
  --m-metadata-column 'frequency of phone cleanage' \
  --o-visualization BGS1014/jaccard_phone_cleaning.qzv \
  --p-pairwise

Saved Visualization to: BGS1014/jaccard_phone_cleaning.qzv
CPU times: user 95.4 ms, sys: 34.2 ms, total: 130 ms
Wall time: 10.5 s


In [10]:
Visualization.load('BGS1014/jaccard_phone_cleaning.qzv')

<visualization: Visualization uuid: b6102c7e-5c6a-4ca7-8c4a-84dc6d4a2c4c>

#### Hand or Phone

In [11]:
%%time
!qiime diversity beta-group-significance \
  --i-distance-matrix alpha_beta1014/jaccard_distance_matrix.qza \
  --m-metadata-file phones_meta3.tsv \
  --m-metadata-column 'hand or phone' \
  --o-visualization BGS1014/jaccard_horp.qzv \
  --p-pairwise

Saved Visualization to: BGS1014/jaccard_horp.qzv
CPU times: user 69.3 ms, sys: 23.5 ms, total: 92.8 ms
Wall time: 8.75 s


In [13]:
Visualization.load('BGS1014/jaccard_horp.qzv')

<visualization: Visualization uuid: 0f4cc8b6-4570-427a-8280-3bd79461bca7>

### Weighted Unifrac

#### PCoA plot

In [14]:
Visualization.load('alpha_beta1014/weighted_unifrac_emperor.qzv')

<visualization: Visualization uuid: 8cac959a-7bbd-4de1-801f-485e013591ed>

#### Gender

In [20]:
%%time
!qiime diversity beta-group-significance \
  --i-distance-matrix alpha_beta1014/weighted_unifrac_distance_matrix.qza \
  --m-metadata-file phones_meta3.tsv \
  --m-metadata-column 'gender' \
  --o-visualization BGS1014/weighted_unifrac_gender.qzv \
  --p-pairwise

Saved Visualization to: BGS1014/weighted_unifrac_gender.qzv
CPU times: user 77.5 ms, sys: 20.6 ms, total: 98.1 ms
Wall time: 9 s


In [21]:
Visualization.load('BGS1014/weighted_unifrac_gender.qzv')

<visualization: Visualization uuid: 407bf16b-9745-45c3-abc5-3a81af4b2f41>

#### Frequency of phone-cleaning

In [22]:
%%time
!qiime diversity beta-group-significance \
  --i-distance-matrix alpha_beta1014/weighted_unifrac_distance_matrix.qza \
  --m-metadata-file phones_meta3.tsv \
  --m-metadata-column 'frequency of phone cleanage' \
  --o-visualization BGS1014/weighted_unifrac_phone_cleaning.qzv \
  --p-pairwise

Saved Visualization to: BGS1014/weighted_unifrac_phone_cleaning.qzv
CPU times: user 74.9 ms, sys: 24.6 ms, total: 99.6 ms
Wall time: 9.24 s


In [23]:
Visualization.load('BGS1014/weighted_unifrac_phone_cleaning.qzv')

<visualization: Visualization uuid: 04cddad2-0ca9-4dd2-8dfa-96ea1e44728a>

#### Hand or Phone

In [24]:
%%time
!qiime diversity beta-group-significance \
  --i-distance-matrix alpha_beta1014/weighted_unifrac_distance_matrix.qza \
  --m-metadata-file phones_meta3.tsv \
  --m-metadata-column 'hand or phone' \
  --o-visualization BGS1014/weighted_unifrac_horp.qzv \
  --p-pairwise

Saved Visualization to: BGS1014/weighted_unifrac_horp.qzv
CPU times: user 80.4 ms, sys: 23.8 ms, total: 104 ms
Wall time: 8.57 s


In [25]:
Visualization.load('BGS1014/weighted_unifrac_horp.qzv')

<visualization: Visualization uuid: cb43b1ab-9c7e-4ba4-9c1c-a6be191439d1>

# -END OF ANALYSIS-

## Further correlation analysis of taxonomic data with Python

The significant decrease in alpha diversity throughout the day suggests that taxa change significantly throughout the day. Usually a drop in diversity is associated with some taxa increasing relative abundance, ie. outcompeting others.

What better way to test this than with a few quick correlation tests in Python through the [Pandas](https://pandas.pydata.org/) packages.

First we'll export the bar-plot data and import with Pandas.

In [58]:
%%time
!qiime tools export \
    --input-path taxonomy/bar-plot.qzv \
    --output-path taxonomy/bar-plot

Exported taxonomy/bar-plot.qzv as Visualization to directory taxonomy/bar-plot
CPU times: user 18.8 ms, sys: 13.7 ms, total: 32.5 ms
Wall time: 1.84 s


In [59]:
# loading pandas package
import pandas as pd
# jupyter magic to display plots inline
%matplotlib inline

In [60]:
# loading data into a Pandas dataframe
# 'time' is set as the row-index column, times are parsed by Python
df = pd.read_csv('taxonomy/bar-plot/level-6.csv',index_col='time',parse_dates=True)

ValueError: 'time' is not in list

In [ ]:
# sorting data by index (inplace to overwrite dataframe)
df.sort_index(inplace=True)
# rows are now sorted by time / sample
df.index

In [ ]:
# provides overview of dataframe
print(df.info())
# shows first couple of rows of dataframe
df.head()

In [ ]:
# data is in absolute counts
# to get relative abundances, the data needs to be normalised
# create a selector for the columns (in this case for the first and last taxonomy column in the dataframe)
start, end = df.columns.get_indexer(['Unassigned;__;__;__;__;__','k__Bacteria;p__Spirochaetes;c__Spirochaetia;o__Spirochaetales;f__Spirochaetaceae;g__Treponema'])
taxa_columns = df.columns[start:end]
df['all'] = df.loc[:,taxa_columns].sum(axis=1)
# divide each taxonomy count by the total value, mulitply by 100
df.loc[:,taxa_columns] = df.loc[:,taxa_columns].div(df['all'],axis=0).mul(100,axis=0)

In [ ]:
df.head()

In [ ]:
# lets create a quick plot to visualise how taxa are changing over time
# the legend is hidden, as it would just cover the whole plot
df.plot(x = 'time-past-brushing-hours', y = taxa_columns, legend=False)

In [ ]:
# It appears that some taxa change quite significantly
# Let's figure out which ones change the most with a few quick operations
# Here we calulate the difference between maximum and mimium for each column and sort them
abund_changes = (df[taxa_columns].max(axis=0)-df[taxa_columns].min(axis=0))
abund_changes.sort_values(ascending=False).head(10)

In [ ]:
# it appears that Streptococcus and Neisseria experience quite some swing throughout the day
# let's see whether their correlation coefficients are also reasonably large
# with the .corrwith function we can correlate several data series (columns) with another
corr_coeff = df[taxa_columns].corrwith(df['time-past-brushing-hours'])
# putting both series into one dataframe for easier intepretation
df2 = pd.concat([abund_changes, corr_coeff],axis=1)
df2.sort_values(by=0,ascending=False).head(10)

In [ ]:
# This looks promising, let's plot some of the data to visualise
df.plot(x='time-past-brushing-hours', y = 'k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Streptococcaceae;g__Streptococcus', kind='line')
df.plot(x='time-past-brushing-hours', y = 'k__Bacteria;p__Proteobacteria;c__Betaproteobacteria;o__Neisseriales;f__Neisseriaceae;g__Neisseria', kind='line')
df.plot(x='time-past-brushing-hours', y = 'k__Bacteria;p__Proteobacteria;c__Betaproteobacteria;o__Burkholderiales;__;__', kind='line')

Simple correlation analysis can be done directly on the dataframe

In [ ]:
# For more a thorough correlation analysis we need to import the stats package from scientific python (scipy)
from scipy import stats

In [ ]:
# we can now carry out individual correlation tests on each of the top 3 taxa
# this will also provide a p-value
R, p = stats.pearsonr(df['k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Streptococcaceae;g__Streptococcus'],df['time-past-brushing-hours'])
print(f'The correlation coefficient is: {R}')
print(f'The p-value is: {p}')
stats.spearmanr(df['k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Streptococcaceae;g__Streptococcus'],df['time-past-brushing-hours'])

In [ ]:
R, p = stats.pearsonr(df['k__Bacteria;p__Proteobacteria;c__Betaproteobacteria;o__Neisseriales;f__Neisseriaceae;g__Neisseria'],df['time-past-brushing-hours'])
print(f'The correlation coefficient is: {R}')
print(f'The p-value is: {p}')
# or a non-parametric correlation test
stats.spearmanr(df['k__Bacteria;p__Proteobacteria;c__Betaproteobacteria;o__Neisseriales;f__Neisseriaceae;g__Neisseria'],df['time-past-brushing-hours'])

In [ ]:
R, p = stats.pearsonr(df['k__Bacteria;p__Proteobacteria;c__Betaproteobacteria;o__Burkholderiales;__;__'],df['time-past-brushing-hours'])
print(f'The correlation coefficient is: {R}')
print(f'The p-value is: {p}')
# or a non-parametric correlation test
stats.spearmanr(df['k__Bacteria;p__Proteobacteria;c__Betaproteobacteria;o__Burkholderiales;__;__'],df['time-past-brushing-hours'])

It appears that there is a strong and significant correlation between *Streptococcus* relative abundance and time after brushing teeth. 

The correlation in *Neisseira* abundance proofed non-significant (p > 0.05) and less strong. Though starting the correlation at a slightly later timepoint would probably provide different results.

Lastly, an unspecified genus from the order *Burkholderiales* also exhibits a strong, signficant correlation

The next step would now be to find biological reasons for these correlations.

**Well done, you are finished with this workshop.** You can now shutdown the notebook, or use the remaining time to explore other [QIIME2 tutorials](https://docs.qiime2.org/2019.7/tutorials/).